In [65]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1"

In [ ]:
wandb.init(project="tesi", name="bigbird")  # args = TrainingArguments(
wandb.stop()

In [9]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import Dataset

modelName = "./robertaLargeIofNil/checkpoint-2178/"

tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForQuestionAnswering.from_pretrained(modelName, return_dict=False).to(
    "cuda:0"
)

In [3]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./nil_el_instanceof.jsonl"
from datasets import Dataset

original_data = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        
        original_data.append(data_line)
print(len(original_data))

7939


In [4]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./aida_train_instanceof.jsonl"
from datasets import Dataset

original_data_aida = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        original_data_aida.append(data_line)
print(len(original_data_aida))

18448


In [5]:
import random
random.shuffle(original_data_aida)
#concat = original_data_aida
concat =original_data_aida[:int(len(original_data_aida)* 0.7)] + original_data


random.shuffle(concat)
#print(len(concat))
train_original = concat[: int(len(concat) * 0.95)]
eval_original = concat[int(len(concat) * 0.95) : ]
dataset = Dataset.from_dict(
    {
        "context": [item["question"] for item in train_original],
        "question": [item["context"] for item in train_original],
        "answers": [item["answers"] for item in train_original],
    }
)
dataset_ev = Dataset.from_dict(
    {
        "context": [item["question"] for item in eval_original],
        "question": [item["context"] for item in eval_original],
        "answers": [item["answers"] for item in eval_original],
    }
)

In [6]:
def get_correct_alignement(context, answer):
    """Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here."""
    gold_text = answer["text"][0]
    
    start_idx = context.find(gold_text)
    end_idx = context.find("</ec>", start_idx)
    return start_idx, end_idx
   


# Tokenize our training dataset
def convert_to_features(example):
    try:
        # Tokenize contexts and questions (as pairs of inputs)

        input_pairs = [example["question"], example["context"]]
        encodings = tokenizer.encode_plus(
            input_pairs, pad_to_max_length=True, truncation="only_second"
        )
        context_encodings = tokenizer.encode_plus(example["context"])

        # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
        # this will give us the position of answer span in the context text

        start_idx, end_idx = get_correct_alignement(
            example["context"], example["answers"]
        )
        if end_idx != -1 and start_idx != -1:
            try:
                start_positions_context = context_encodings.char_to_token(start_idx)
                end_positions_context = context_encodings.char_to_token(end_idx)

                # here we will compute the start and end position of the answer in the whole example
                # as the example is encoded like this <s> question</s></s> context</s>
                # and we know the postion of the answer in the context
                # we can just find out the index of the sep token and then add that to position + 1 (+1 because there are two sep tokens)
                # this will give us the position of the answer span in whole example
                sep_idx = encodings["input_ids"].index(tokenizer.sep_token_id)
                start_positions = start_positions_context + sep_idx
                end_positions = end_positions_context + sep_idx + 1
               
                encodings.update(
                    {
                        "start_positions": start_positions,
                        "end_positions": end_positions,
                        "attention_mask": encodings["attention_mask"],
                    }
                )
                return encodings
            except Exception as e:
                print(e)
        else:
            print("qu")
            encodings.update(
                {
                    "start_positions": start_idx,
                    "end_positions": end_idx,
                    "attention_mask": encodings["attention_mask"],
                    "is_nil": 1 if example["answers"]["text"][0] == "Not In Candidates" else 0
                }
            )
            return encodings
    except Exception as e:
        print("errors", e)

In [ ]:
train_ds = dataset.map(convert_to_features)
eval_ds = dataset_ev.map(convert_to_features) 

In [ ]:
import joblib
import datasets

In [ ]:
joblib.dump( train_ds, './dumps/aidaTrainIOF.dump',)
joblib.dump( eval_ds, './dumps/aidaEvalIOF.dump',)

In [ ]:
train_ds = joblib.load('./dumps/aidaTrainNERBigBird.dump')
eval_ds = joblib.load('./dumps/aidaEvalNERBigBird.dump')

In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

import wandb
torch.cuda.set_device(0)
torch.cuda.current_device()


args = TrainingArguments(
    "robertaLargefNil",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",
    load_best_model_at_end=False,
    logging_strategy="steps",
    logging_steps=500,
    eval_steps=500,
    save_steps=500,
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=100,
    num_train_epochs=14,
    weight_decay=0.02,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using amp fp16 backend


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: answers, context, question.
***** Running training *****
  Num examples = 19809
  Num Epochs = 14
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 100
  Total optimization steps = 2772
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: agazzi-ruben99 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.16.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,Training Loss,Validation Loss
0,No log,0.386881
1,No log,0.337372
2,0.466700,0.329125
3,0.466700,0.356657
4,0.466700,0.387810


/opt/miniconda3/envs/extend/lib/python3.9/site-packages/transformers/trainer.py:1355: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: answers, context, question.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 1
Saving model checkpoint to robertaLargefNil/checkpoint-198
Configuration saved in robertaLargefNil/checkpoint-198/config.json
Model weights saved in robertaLargefNil/checkpoint-198/pytorch_model.bin
tokenizer config file saved in robertaLargefNil/checkpoint-198/tokenizer_config.json
Special tokens file saved in robertaLargefNil/chec

In [ ]:
trainer.save_model('./RoBertAAida')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dataset[0]

## Prediction and evaluation

In [10]:
def check_brackets(input_string):
    stack = []
    brackets = {"(": ")", "{": "}", "[": "]"}
    for char in input_string:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if not stack or brackets[stack.pop()] != char:
                return False
    return not stack


def process_answer(answer, candidates):
    if answer == "Not In Candidates":
        return answer
    else:
        modified_answer = answer.split("</ec>")[0]
        modified_answer = modified_answer.split(": instance of ")[0]
        modified_answer = modified_answer.split(": instance")[0]
        modified_answer = modified_answer.replace("<s>", "")
        modified_answer = modified_answer.replace("</ec", "")
        modified_answer = modified_answer.replace("</s>", "")
        modified_answer = modified_answer.replace("<s", "")
        modified_answer = modified_answer.replace("</", "")
        modified_answer = modified_answer.replace(" ec", "")
        modified_answer = modified_answer.replace(">", "")
        modified_answer = modified_answer.strip()
        if not check_brackets(modified_answer):
            modified_answer = modified_answer.replace("(", "")
            modified_answer = modified_answer.replace(")", "")
        if modified_answer == "Not In Candidates":
            modified_answer = "Not In Candidates"
        if modified_answer == "":
            modified_answer = "Not In Candidates"
        if modified_answer not in candidates:
            modified_answer = "Not In Candidates"
        return modified_answer


process_answer("> Alzenau </", ["Alzenau"])

'Alzenau'

In [3]:
import gc
import torch.nn.functional as F
import random
import numpy as np


def get_context(text, n=200):
    start_ent = "[START_ENT]"
    end_ent = "[END_ENT]"

    start_idx = text.find(start_ent)
    end_idx = text.find(end_ent) + len(end_ent)

    before_start_ent = text[max(0, start_idx - n) : start_idx]
    mention = text[start_idx:end_idx]
    after_end_ent = text[end_idx : end_idx + n]
    return before_start_ent + mention + after_end_ent


def entropy(p):
    return -np.sum(p * np.log2(p))


def make_prediction(data_entry, nil_prediction):
    with torch.no_grad():
        question = data_entry["input"]
        context = ""
        # if nil_prediction:
        #     context += " Not In Candidates </ec> "
        index = 0
        added = False
        candidates = data_entry["candidates"]
        # random.shuffle(candidates)
        for item in candidates:
            context += item + f" </ec> "
            index += 1
            if index == 1 and nil_prediction:
                context += " Not In Candidates </ec> "
                added = True
        if not added and nil_prediction:
            context = " Not In Candidates </ec> " + context
        input_pairs = [question, context]

        encodings = tokenizer.encode_plus(
            input_pairs, return_tensors="pt", truncation="only_second"
        ).to("cuda")
        if len(encodings["input_ids"][0]) > 512:
            print(len(data_entry["candidates"]))
        start_scores, end_scores = model(
            encodings["input_ids"],
            attention_mask=encodings["attention_mask"],
        )
        start_scores = F.softmax(start_scores, dim=1)
        end_scores = F.softmax(end_scores, dim=1)
        startEntropy = entropy(start_scores[0].cpu().numpy())
        endEntropy = entropy(end_scores[0].cpu().numpy())
        # calculate mean entropy
        meanEntropy = (startEntropy + endEntropy) / 2
        # start_scores.to("cpu")
        # end_scores.to("cpu")
        start = torch.argmax(
            start_scores
        )  # Get the most likely beginning of answer with the argmax of the score
        end = (
            torch.argmax(end_scores) + 1
        )  # Get the most likely end of answer with the argmax of the score
        mean_score = 0
        try:
            mean_score = end_scores[0][end.item()] + start_scores[0][start.item()]
        except:
            None
        answer_tokens = encodings["input_ids"][0, start.item() : end.item() + 1]
        answer = ""

        answer = process_answer(tokenizer.decode(answer_tokens), context)
        classified = 0
        if (meanEntropy > 0.5785785785785785) and nil_prediction:
            answer = "Not In Candidates"
        else:
            if answer == "":
                answer = "Not In Candidates"
        del encodings
        del end_scores
        del start_scores
        del start
        del end
        score = float(mean_score)
        gc.collect()
        torch.cuda.empty_cache()
        return {
            "correct": data_entry["output"][0]["answer"],
            "non_processed": tokenizer.decode(answer_tokens),
            "predicted": answer,
            "input_phrase": question,
            "scores": score,
            "candidates": context,
            "entropy": meanEntropy,
        }

In [ ]:
import wandb
wandb.init()

In [11]:
from tqdm import tqdm
import json
import torch

results = []
file_path = "./nil_el_test_instanceof.jsonl"
dataset = []
with open(file_path, "r") as file:
    # dataset = json.load(file)
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)

        dataset.append(data_line)

for item in tqdm(dataset):
    try:
        pred = make_prediction(item, True)
        results.append(pred)
    except Exception as e:
        print(e)

  0%|          | 0/992 [00:00<?, ?it/s]

100%|██████████| 992/992 [01:46<00:00,  9.34it/s]


In [10]:
results[0]

{'correct': 'New York City',
 'non_processed': '> New York City : instance of city in the United States </ec',
 'predicted': 'New York City',
 'input_phrase': 'Stocks end back and forth day slightly higher [START_ENT] New York [END_ENT] Stocks finished an erratic session mixed Tuesday as higher commodity prices lifted energy and materials shares Major stock indexes had their third straight advance reaching new 13 month highs but the',
 'scores': 0.9999992847442627,
 'candidates': 'New York : instance of Wikimedia disambiguation page </ec>  Not In Candidates </ec> New York City : instance of city in the United States </ec> New York (magazine) : instance of magazine </ec> United States congressional delegations from New York : instance of Wikimedia list article </ec> New York (film) : instance of Unknown </ec> Province of New York : instance of crown colony </ec> Manhattan : instance of borough of New York City </ec> New York Knicks : instance of basketball team </ec> New York metropolit

In [12]:
correct = 0
correct_trace = []
wrong = []
correct_nil = 0
wrong_nil = 0
full_data = []
for result in tqdm(results):
    processed = result["predicted"]
    if processed == result["correct"]:
        correct += 1
        correct_trace.append(result)
        if processed == "Not In Candidates":
            correct_nil += 1
    else:
        if result["correct"] == "Not In Candidates":
            wrong_nil += 1
        wrong.append(result)
    full_data.append(
        {
            "score": result["scores"],
            "nil": int(result["correct"] == "Not In Candidates"),
        }
    )

100%|██████████| 992/992 [00:00<00:00, 476658.22it/s]


In [13]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.6381048387096774
accuracy in nil prediction is: 0.8308157099697885


zeshel acc = 0.4676229922214365

In [33]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.836890243902439
accuracy in nil prediction is: 0.5935483870967742
